In [4]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SEED_BKG_DIR = '/data/darknet/python/background/'
GENE_IMG_DIR = '/data/darknet/python/images_nanchangperson_replaced_bkg/'


def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)

seed_xml_names.sort()
for xml_name in seed_xml_names:
    xml_name_part = xml_name[-15:-4]
    print(xml_name_part)
    if not (int(xml_name_part) >= 420000):
        continue
    background_img_data = cv2.imread(SEED_BKG_DIR + seed_bkg_names[0])
    print(background_img_data.shape)
    
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    print(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', bboxes, background_img_data)

00000420000
(480, 640, 3)
/data/darknet/python/image/00000420000.jpg
00000420001
(480, 640, 3)
/data/darknet/python/image/00000420001.jpg
00000420002
(480, 640, 3)
/data/darknet/python/image/00000420002.jpg
00000420003
(480, 640, 3)
/data/darknet/python/image/00000420003.jpg
00000420004
(480, 640, 3)
/data/darknet/python/image/00000420004.jpg
00000420005
(480, 640, 3)
/data/darknet/python/image/00000420005.jpg
00000420006
(480, 640, 3)
/data/darknet/python/image/00000420006.jpg
00000420007
(480, 640, 3)
/data/darknet/python/image/00000420007.jpg
00000420008
(480, 640, 3)
/data/darknet/python/image/00000420008.jpg
00000420009
(480, 640, 3)
/data/darknet/python/image/00000420009.jpg
00000420010
(480, 640, 3)
/data/darknet/python/image/00000420010.jpg
00000420011
(480, 640, 3)
/data/darknet/python/image/00000420011.jpg
00000420012
(480, 640, 3)
/data/darknet/python/image/00000420012.jpg
00000420013
(480, 640, 3)
/data/darknet/python/image/00000420013.jpg
00000420014
(480, 640, 3)
/data/da

00000420120
(480, 640, 3)
/data/darknet/python/image/00000420120.jpg
00000420121
(480, 640, 3)
/data/darknet/python/image/00000420121.jpg
00000420122
(480, 640, 3)
/data/darknet/python/image/00000420122.jpg
00000420123
(480, 640, 3)
/data/darknet/python/image/00000420123.jpg
00000420124
(480, 640, 3)
/data/darknet/python/image/00000420124.jpg
00000420125
(480, 640, 3)
/data/darknet/python/image/00000420125.jpg
00000420126
(480, 640, 3)
/data/darknet/python/image/00000420126.jpg
00000420127
(480, 640, 3)
/data/darknet/python/image/00000420127.jpg
00000420128
(480, 640, 3)
/data/darknet/python/image/00000420128.jpg
00000420129
(480, 640, 3)
/data/darknet/python/image/00000420129.jpg
00000420130
(480, 640, 3)
/data/darknet/python/image/00000420130.jpg
00000420131
(480, 640, 3)
/data/darknet/python/image/00000420131.jpg
00000420132
(480, 640, 3)
/data/darknet/python/image/00000420132.jpg
00000420133
(480, 640, 3)
/data/darknet/python/image/00000420133.jpg
00000420134
(480, 640, 3)
/data/da

(480, 640, 3)
/data/darknet/python/image/00000420239.jpg
00000420240
(480, 640, 3)
/data/darknet/python/image/00000420240.jpg
00000420241
(480, 640, 3)
/data/darknet/python/image/00000420241.jpg
00000420242
(480, 640, 3)
/data/darknet/python/image/00000420242.jpg
00000420243
(480, 640, 3)
/data/darknet/python/image/00000420243.jpg
00000420244
(480, 640, 3)
/data/darknet/python/image/00000420244.jpg
00000420245
(480, 640, 3)
/data/darknet/python/image/00000420245.jpg
00000420246
(480, 640, 3)
/data/darknet/python/image/00000420246.jpg
00000420247
(480, 640, 3)
/data/darknet/python/image/00000420247.jpg
00000420248
(480, 640, 3)
/data/darknet/python/image/00000420248.jpg
00000420249
(480, 640, 3)
/data/darknet/python/image/00000420249.jpg
00000420250
(480, 640, 3)
/data/darknet/python/image/00000420250.jpg
00000420251
(480, 640, 3)
/data/darknet/python/image/00000420251.jpg
00000420252
(480, 640, 3)
/data/darknet/python/image/00000420252.jpg
00000420253
(480, 640, 3)
/data/darknet/python

00000420359
(480, 640, 3)
/data/darknet/python/image/00000420359.jpg
00000420360
(480, 640, 3)
/data/darknet/python/image/00000420360.jpg
00000420361
(480, 640, 3)
/data/darknet/python/image/00000420361.jpg
00000420362
(480, 640, 3)
/data/darknet/python/image/00000420362.jpg
00000420363
(480, 640, 3)
/data/darknet/python/image/00000420363.jpg
00000420364
(480, 640, 3)
/data/darknet/python/image/00000420364.jpg
00000420365
(480, 640, 3)
/data/darknet/python/image/00000420365.jpg
00000420366
(480, 640, 3)
/data/darknet/python/image/00000420366.jpg
00000420367
(480, 640, 3)
/data/darknet/python/image/00000420367.jpg
00000420368
(480, 640, 3)
/data/darknet/python/image/00000420368.jpg
00000420369
(480, 640, 3)
/data/darknet/python/image/00000420369.jpg
00000420370
(480, 640, 3)
/data/darknet/python/image/00000420370.jpg
00000420371
(480, 640, 3)
/data/darknet/python/image/00000420371.jpg
00000420372
(480, 640, 3)
/data/darknet/python/image/00000420372.jpg
00000420373
(480, 640, 3)
/data/da

/data/darknet/python/image/00000420487.jpg
00000420488
(480, 640, 3)
/data/darknet/python/image/00000420488.jpg
00000420489
(480, 640, 3)
/data/darknet/python/image/00000420489.jpg
00000420490
(480, 640, 3)
/data/darknet/python/image/00000420490.jpg
00000420491
(480, 640, 3)
/data/darknet/python/image/00000420491.jpg
00000420492
(480, 640, 3)
/data/darknet/python/image/00000420492.jpg
00000420493
(480, 640, 3)
/data/darknet/python/image/00000420493.jpg
00000420494
(480, 640, 3)
/data/darknet/python/image/00000420494.jpg
00000420495
(480, 640, 3)
/data/darknet/python/image/00000420495.jpg
00000420496
(480, 640, 3)
/data/darknet/python/image/00000420496.jpg
00000420497
(480, 640, 3)
/data/darknet/python/image/00000420497.jpg
00000420498
(480, 640, 3)
/data/darknet/python/image/00000420498.jpg
00000420499
(480, 640, 3)
/data/darknet/python/image/00000420499.jpg
00000420500
(480, 640, 3)
/data/darknet/python/image/00000420500.jpg
00000420501
(480, 640, 3)
/data/darknet/python/image/0000042

...

(480, 640, 3)
/data/darknet/python/image/00000420867.jpg
00000420868
(480, 640, 3)
/data/darknet/python/image/00000420868.jpg
00000420869
(480, 640, 3)
/data/darknet/python/image/00000420869.jpg
00000420870
(480, 640, 3)
/data/darknet/python/image/00000420870.jpg
00000420871
(480, 640, 3)
/data/darknet/python/image/00000420871.jpg
00000420872
(480, 640, 3)
/data/darknet/python/image/00000420872.jpg
00000420873
(480, 640, 3)
/data/darknet/python/image/00000420873.jpg
00000420874
(480, 640, 3)
/data/darknet/python/image/00000420874.jpg
00000420875
(480, 640, 3)
/data/darknet/python/image/00000420875.jpg
00000420876
(480, 640, 3)
/data/darknet/python/image/00000420876.jpg
00000420877
(480, 640, 3)
/data/darknet/python/image/00000420877.jpg
00000420878
(480, 640, 3)
/data/darknet/python/image/00000420878.jpg
00000420879
(480, 640, 3)
/data/darknet/python/image/00000420879.jpg
00000420880
(480, 640, 3)
/data/darknet/python/image/00000420880.jpg
00000420881
(480, 640, 3)
/data/darknet/python

(480, 640, 3)
/data/darknet/python/image/00000420994.jpg
00000420995
(480, 640, 3)
/data/darknet/python/image/00000420995.jpg
00000420996
(480, 640, 3)
/data/darknet/python/image/00000420996.jpg
00000420997
(480, 640, 3)
/data/darknet/python/image/00000420997.jpg
00000420998
(480, 640, 3)
/data/darknet/python/image/00000420998.jpg
00000420999
(480, 640, 3)
/data/darknet/python/image/00000420999.jpg
00000421000
(480, 640, 3)
/data/darknet/python/image/00000421000.jpg
00000421001
(480, 640, 3)
/data/darknet/python/image/00000421001.jpg
00000421002
(480, 640, 3)
/data/darknet/python/image/00000421002.jpg
00000421003
(480, 640, 3)
/data/darknet/python/image/00000421003.jpg
00000421004
(480, 640, 3)
/data/darknet/python/image/00000421004.jpg
00000421005
(480, 640, 3)
/data/darknet/python/image/00000421005.jpg
00000421006
(480, 640, 3)
/data/darknet/python/image/00000421006.jpg
00000421007
(480, 640, 3)
/data/darknet/python/image/00000421007.jpg
00000421008
(480, 640, 3)
/data/darknet/python

(480, 640, 3)
/data/darknet/python/image/00000421119.jpg
00000421120
(480, 640, 3)
/data/darknet/python/image/00000421120.jpg
00000421121
(480, 640, 3)
/data/darknet/python/image/00000421121.jpg
00000421122
(480, 640, 3)
/data/darknet/python/image/00000421122.jpg
00000421123
(480, 640, 3)
/data/darknet/python/image/00000421123.jpg
00000421124
(480, 640, 3)
/data/darknet/python/image/00000421124.jpg
00000421125
(480, 640, 3)
/data/darknet/python/image/00000421125.jpg
00000421126
(480, 640, 3)
/data/darknet/python/image/00000421126.jpg
00000421127
(480, 640, 3)
/data/darknet/python/image/00000421127.jpg
00000421128
(480, 640, 3)
/data/darknet/python/image/00000421128.jpg
00000421129
(480, 640, 3)
/data/darknet/python/image/00000421129.jpg
00000421130
(480, 640, 3)
/data/darknet/python/image/00000421130.jpg
00000421131
(480, 640, 3)
/data/darknet/python/image/00000421131.jpg
00000421132
(480, 640, 3)
/data/darknet/python/image/00000421132.jpg
00000421133
(480, 640, 3)
/data/darknet/python

00000421246
(480, 640, 3)
/data/darknet/python/image/00000421246.jpg
00000421247
(480, 640, 3)
/data/darknet/python/image/00000421247.jpg
00000421248
(480, 640, 3)
/data/darknet/python/image/00000421248.jpg
00000421249
(480, 640, 3)
/data/darknet/python/image/00000421249.jpg
00000421250
(480, 640, 3)
/data/darknet/python/image/00000421250.jpg
00000421251
(480, 640, 3)
/data/darknet/python/image/00000421251.jpg
00000421252
(480, 640, 3)
/data/darknet/python/image/00000421252.jpg
00000421253
(480, 640, 3)
/data/darknet/python/image/00000421253.jpg
00000421254
(480, 640, 3)
/data/darknet/python/image/00000421254.jpg
00000421255
(480, 640, 3)
/data/darknet/python/image/00000421255.jpg
00000421256
(480, 640, 3)
/data/darknet/python/image/00000421256.jpg
00000421257
(480, 640, 3)
/data/darknet/python/image/00000421257.jpg
00000421258
(480, 640, 3)
/data/darknet/python/image/00000421258.jpg
00000421259
(480, 640, 3)
/data/darknet/python/image/00000421259.jpg
00000421260
(480, 640, 3)
/data/da

(480, 640, 3)
/data/darknet/python/image/00000421370.jpg
00000421371
(480, 640, 3)
/data/darknet/python/image/00000421371.jpg
00000421372
(480, 640, 3)
/data/darknet/python/image/00000421372.jpg
00000421373
(480, 640, 3)
/data/darknet/python/image/00000421373.jpg
00000421374
(480, 640, 3)
/data/darknet/python/image/00000421374.jpg
00000421375
(480, 640, 3)
/data/darknet/python/image/00000421375.jpg
00000421376
(480, 640, 3)
/data/darknet/python/image/00000421376.jpg
00000421377
(480, 640, 3)
/data/darknet/python/image/00000421377.jpg
00000421378
(480, 640, 3)
/data/darknet/python/image/00000421378.jpg
00000421379
(480, 640, 3)
/data/darknet/python/image/00000421379.jpg
00000421380
(480, 640, 3)
/data/darknet/python/image/00000421380.jpg
00000421381
(480, 640, 3)
/data/darknet/python/image/00000421381.jpg
00000421382
(480, 640, 3)
/data/darknet/python/image/00000421382.jpg
00000421383
(480, 640, 3)
/data/darknet/python/image/00000421383.jpg
00000421384
(480, 640, 3)
/data/darknet/python